## 4.6 Database Link in Python

### psycopg2

설치
```cmd
pip install psycopg2
pip install psycopg2-binary #위의 설치 에러 시 이 코드 실행
```
<br>

사용법
```python
conn = psycopg2.connect(
  host='address',
  dbname='database name',
  user='user name',
  password='password',
  port=port)

cur = conn.cursor()
```

CRUD
```python
cur.execute(f'INSERT INTO test (id, press_name) VALUES ({id}, {press_name});')
conn.commit() # CREATE

cur.execute('SELECT * FROM test;')
result_one = cur.fetchone() # READ
result_many = cur.fetchmany() 
result_all = cur.fetchall() 

cur.execute(f'UPDATE test SET press_name={press_name} WHERE id > 5')
conn.commit() # UPDATE

cur.execute('DELETE FROM test WHERE press_name LIKE %조선%;')
conn.commit() # DELETE
```
<br>

pandas 내의 method를 통해서도 사용 가능
```python
pd.read_sql('SELECT * FROM test', conn)
```
<br>

Closer <br>
사용 후 연결 해제 <br>
```python
cur.close()
conn.close()
```
<br>

아래와 같이 사용 가능
```python
with conn.cursor() as cur:
  cur.execute(query)

conn.close()

with psycopg2.connect():
  with conn.cursor() as cur:
    cur.execute(query)
```

In [41]:
import psycopg2

In [42]:
conn = psycopg2.connect(
  host='localhost',
  dbname='postgres',
  user='postgres',
  password='postgrespw',
  port=49153)

cur = conn.cursor()

In [43]:
cur.execute('SELECT * FROM news_id')
#result_one = cur.fetchone() # 1개
#result_many = cur.fetchmany(2) # 가져올 개수
result_all = cur.fetchall()

In [44]:
result_all

[('020', '동아일보'), ('009', '매일경제'), ('023', '조선일보')]

pandas로 sql read

In [45]:
import pandas as pd

In [46]:
pd.read_sql('SELECT * FROM news_id', conn)

,id,press_name
0,020,동아일보
1,009,매일경제
2,023,조선일보


### sqlalchemy

설치
```cmd
pip install sqlalchemy
```
<br>

사용법
```python
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime

db_url = f'postgres+psycopg2://{USERNAME}:{PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}'
engine = create_engine(db_url, echo=True)

test.to_sql(
  'schema',
  engine,
  if_exists='append', # replace: 덮어쓰기
  index=False,
  chunksize=5000,
  dtypes={
    'id': Integer,
    'press_name': Text
  }
)
```

In [47]:
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime

In [48]:
USERNAME = 'postgres'
PASSWORD = 'postgrespw'
DB_HOST = 'localhost'
PORT = '49153'
DB_NAME = 'postgres'

db_url = f'postgresql://{USERNAME}:{PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}'
engine = create_engine(db_url, echo=True)


# 5.Crawling

스크레이핑(Scraping)이라고도 하며 웹 페이지 내의 데이터를 추출하는 것을 의미 <br>
데이터를 수집하기 위한 방법으로 많이 사용 <br>
크게 두 가지의 방법이 존재
1. 정적크롤링 <br>
정적 데이터를 수집하는 방법 <br>
정적 데이터란 페이지 내에 원하는 정보가 모두 드러남

2. 동적크롤링 <br>
동적 데이터를 수집하는 방법 <br>
동적 데이터란 클릭, 로그인 등의 행위를 통해 원하는 데이터에 접근 가능 <br>


|    |정적 크롤링|동적 크롤링|
|----|--------|--------|
|방법 |주소 사용  |브라우저 사용|
|수집 범위|제한적  |제한 없음|
|속도|매우 빠름|매우 느림
<br>

크롤링 시 사이트에서 크롤링을 허용하는지를 반드시 확인해야 함 <br>
robots.txt를 뒤에 붙여 확인 가능 <br>
강제는 아니나 이를 무시하면 추후 법률적 문제가 생길 수 있음
```
www.daum.net/robots.txt
User-agent: *
Disallow: /
```
*: All <br>
/: All Directories


## 5.1 HTTP

WWW(World Wide Web, W3) 상에서 정보를 주고받을 수 있는 프로토콜 <br>
클라이언트와 서버 사이에 이루어지는 요청/응답 프로토콜

### API
API(Application Programming Interface) <br>
- Application: 고유한 기능을 가진 모든 소프트웨어
- Interface: 두 애플리케이션 간의 규약 <br>
이 계약은 요청과 응답을 사용하여 두 애플리케이션이 서로 통신하는 방법을 정의합니다.

### REST
REST(Representational State Transfer): 자원을 이름으로 구분하여 해당 자원의 상태를 주고받는 것 <br>

REST 구성
1. 자원(Resource): HTTP URI
2. 자원에 대한 행위(Verb): HTTP Method
3. 자원에 대한 행위의 내용(Representations): HTTP Message Pay Load


- HTTP URI(Uniform Resource Identifier)를 통해 자원(Resource)을 명시
- HTTP Method(POST, GET, PUT, DELETE)를 사용하여 URI에 대한 CRUD Operation을 적용 <br>
<br>

HTTP Methods
- GET: 자원 검색
- POST: 자원 작성
- PUT: 자원 업데이트
- DELETE: 데이터 삭제
- HEAD: 자원 검색 (GET과 유사하나 상태 줄과 헤더만 반환)
- OPTIONS: 자원이 지원하고 있는 메소드의 취득
- PATCH: 자원 일부 수정 (PUT과 유사하나 일부만 수정)
- CONNECT: 자원의 터널 접속을 변경
- TRACE: 리소스에 대한 경로를 따라 메시지 루프백 테스트를 수행
<br>

HTTP Status
1. 1xx(Informational): 요청 처리중
2. 2xx(Successful): 요청 정상 처리 <br>
200: 요청 성공
3. 3xx(Redirection): 요청을 완료하려면 추가 행동이 필요
4. 4xx(Client Error): 클라이언트 오류, 잘못된 문법등으로 요청을 수행할 수 없음 <br>
400: Bad Request, 클라이언트의 잘못된 요청으로 서버가 요청을 처리할 수 없음 <br>
401: Unauthorized, 해당 리소스에 대한 인증이 필요함 <br>
403: Forbidden, 서버가 요청을 이해했지만 승인을 거부함 <br>
404: Not Found, 리소스를 찾을 수 없음 <br>
5. 5xx(Server Error): 서버 오류

### REST API
REST의 원리를 따르는 API <br>
※ RESTful: REST의 원리를 따르는 시스템

## 5.2 HTML
HTML(HyperText Markup Language)은 웹 페이지 표시를 위해 개발된 지배적인 마크업 언어 <br>
HTML은 웹 페이지 콘텐츠 안의 꺾쇠 괄호에 둘러싸인 "태그"로 되어있는 HTML 요소 형태로 작성 <br>
HTML은 웹 브라우저와 같은 HTML 처리 장치의 행동에 영향을 주는 자바스크립트, 본문과 그 밖의 항목의 외관과 배치를 정의하는 CSS 같은 스크립트를 포함하거나 불러올 수 있음 <br>
<br>
HTML 선택자: HTML에서는 다수의 동일한 태그가 존재하는데 각 태그를 구별할 수 있도록 선택자를 이용
```html
<div> 
	<div> 
      <a> c </a> 
      <span> c++ </span> 
    </div> 
    
    <div> 
      <a> java </a> 
      <span> python </span> 
    </div> 
</div>
```

```html
<div id="contents"> 
	<div class="data1"> 
      <span class="language"> c++ </span> 
      <span class="language"> java </span> 
      <span class="language"> python </span> 
  </div> 
    
  <div class="data2"> 
      <a class="framework"> tensorflow </a> 
      <a class="framework"> pytorch </a> 
      <a class="framework"> spring </a> 
  </div> 
</div>
```

## 5.3 정적크롤링

#### 5.3.1 라이브러리

##### 5.3.1.1 requests

requests: 파이썬용 http 라이브러리 <br>
reference: https://requests.readthedocs.io/en/latest/

메소드별 사용법
```python
GET: requests.get()
POST: requests.post()
PUT: requests.put()
DELETE: requests.delete()
```

```python
import requests

requests.get("https://jsonplaceholder.typicode.com/users/1")
```

###### Response Body

요청이 정상적으로 처리가 되면, response body에 요청한 데이터가 담겨져 옴. <br>
response body 크게 3가지 방식으로 읽을 수 있음 <br>
1. content: binary 원문을 읽음
```python
response.content
```
2. text: utf-8로 인코딩 된 문자열로 읽음
```python
response.text
```
3. json: 응답이 json이면 dict로 읽음
```python
response.json()
```



###### Request

param: 주소에 포함된 변수를 담음<br>
ex) https://www.naver.com/post/12345 <br>
-> 12345 <br>
query: 주소 바깥? 이후의 주소를 담음<br>
ex) https://www.naver.com/post/post_id=12345&id=1 <br>
-> 12345, 1
body: XML, JSON 등의 데이터를 담음, 주소에서는 확인 불가<br>
<br>

requests에서는 아래와 같이 사용
- get
```python
response = requests.get("https://naver.com/post", params={"post_id": "12345", "id": "1"})
```

- post, put: HTML 데이터 전송
```python
response = requests.get("https://naver.com/post", data={"post_id": "12345", "id": "1"})
```
json 형태로도 요청 가능
```python
response = requests.get("https://naver.com/post", json={"post_id": "12345", "id": "1"})
```

###### headers

일부 웹 사이트는 bot agent를 차단 <br>
이 경우 header의 user-agent를 아래와 같이 넘기면 해결 <br>
```python
requests.get("https://naver.com/post", headers={'User-Agent': 'Mozilla 5.0'})
```

##### 5.3.1.2 BeautifulSoup

BeautifulSoup: html, xml 등으로부터 원하는 정보를 가지고 올 수 있도록 하는 라이브러리 <br>
reference: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

```python
import requests
from bs4 import BeautifulSoup

response = requests.get(url)
bs = BeautifulSoup(response.text, 'lxml')
```

###### Parser

|parser|특징|설치|속도|사용방법|
|------|---|---|---|------|
|html.parser||기본|보통|BeautifulSoup(html_doc, 'html.parser')|
|lxml|xml 지원|lxml 필요|빠름|BeautifulSoup(html_doc, 'lxml')|
|xml|xml 지원|lxml 필요|빠름|BeautifulSoup(html_doc, 'xml')|
|html5lib|브라우저와 동일|html5lib 필요|느림|BeautifulSoup(html_doc, 'html5lib')|

###### find
속성과 값을 이용하여 원하는 값을 찾음

find: 매칭되는 값 중 상위 1개 반환
find_all: 매칭되는 전체 반환

특정 태그 추출
```python
soup.find_all('p') # p 태그 추출
```
<br>

특정 클래스 추출
```python
soup.find_all(class_='a') # a 클래스 추출
```
<br>

특정 태그와 class 추출
```python
soup.find_all('p', attrs={'class': 'a'}) # p 태그와 a 클래스 모두를 갖는 값 추출
```
<br>

특정 id 추출
```python
soup.find_all(id='b') # b id를 갖는 값 추출
```
<br>

###### select
CSS Selector로 태그를 찾아 반환 <br>
CSS에서 HTML을 태깅하는 방법을 활용 <br>
<br>
select_one: 매칭되는 값 중 상위 1개 반환 <br>
select: 매칭되는 전체 반환 <br>
<br>

특정 태그 추출
```python
soup.select('p') # p 태그 추출
```
<br>

특정 클래스 추출
```python
soup.select('.a') # a 클래스 추출
```
<br>

특정 태그와 class 추출
```python
soup.select('p.a') # p 태그와 a 클래스 모두를 갖는 값 추출
```
<br>

특정 id 추출
```python
soup.select('#b') # b id를 갖는 값 추출
```
<br>

특정 태그와 id 추출
```python
soup.select('p#b') # p 태그와 b id 모두를 갖는 값 추출
```
<br>

특정 태그와 class, id 모두 추출
```python
soup.select('p.a#b') # p 태그와 a 클래스 b id 모두 갖는 값 추출
```
<br>

특정 태그 아래에 있는 태그 찾기
```python
soup.select('div p') # div 아래 p태그가 있는 값 추출
soup.select('div > p') # div 바로 아래 p태그가 있는 값 추출
soup.select("div > #link") # div 바로 아래 link id가 있는 값 추출
```
<br>

형제 태그 찾기
```python
soup.select("#link + .sister") # link 태그와 형제 태그 중 바로 직후 1개
soup.select("#link ~ .sister") # link 태그와 형제 태그 중 뒤에 태그 전부
```
<br>

여러 태그 중 i번째 태그 추출
```python
soup.select('a:nth-of-type(i)') # 추출된 a태그 중 i번째 값 반환
```
<br>
<br>

정규표현식 활용 <br>
```python
soup.select('[class~=a]') # class 속성 중 a를 포함하는 태그
soup.select('a[href]') # a 태그 중 href 속성이 존재하는 태그
soup.select('a[href="https://www.naver.com"]') # a 태그 중 href 속성이 https://www.naver.com과 매칭되는 태그
soup.select('a[href^="https://"]') # a 태그 중 href 속성이 https://로 시작하는 태그
soup.select('a[href$="ac.kr"]') # a 태그 중 href 속성이 ac.kr로 끝나는 태그
soup.select('a[href*="naver"]') # a 태그 중 href 속성 중 naver를 가지는 태그
```

<br>
<br>

출력
```python
soup.strings # 값 반환
soup.stripped_strings # 공백을 제거한 값 반환
```

### 5.3.2 실습

##### 예제

```html
<div id="contents"> 
    <div class="data1"> 
      <span class="language"> c++ </span> 
      <span class="language"> java </span> 
      <span class="language"> python </span> 
  </div> 

  <div class="data2"> 
      <a class="framework"> tensorflow </a> 
      <a class="framework"> pytorch </a> 
      <a class="framework"> spring </a> 
  </div> 
</div>
```

In [10]:
response = '''<div id="contents"> 
	<div class="data1"> 
      <span class="language"> c++ </span> 
      <span class="language"> java </span> 
      <span class="language"> python </span> 
  </div> 
    
  <div class="data2"> 
      <a class="framework"> tensorflow </a> 
      <a class="framework"> pytorch </a> 
      <a class="framework"> spring </a> 
  </div> 
</div>'''

In [11]:
from bs4 import BeautifulSoup

In [12]:
bs = BeautifulSoup(response, 'lxml')

In [13]:
bs.select('div')[0].select('a') # a태그
bs.select('div')[0].select('.framework') # 클래스 framework
bs.select('div')[0].select('a.framework') # a태그 중 클래스값 framework

[<a class="framework"> tensorflow </a>,
 <a class="framework"> pytorch </a>,
 <a class="framework"> spring </a>]

In [14]:
bs.select('#contents') # id 가져오기
bs.select('div#contents') # div 태그 contents라는 id 가져오기

[<div id="contents">
 <div class="data1">
 <span class="language"> c++ </span>
 <span class="language"> java </span>
 <span class="language"> python </span>
 </div>
 <div class="data2">
 <a class="framework"> tensorflow </a>
 <a class="framework"> pytorch </a>
 <a class="framework"> spring </a>
 </div>
 </div>]

In [15]:
# span태그에서 java 값만 추출하기
bs.select('span')[1] 
bs.select('span:nth-of-type(2)')

[<span class="language"> java </span>]

In [16]:
bs.select('div a') #div태그 아래에 있는 a태그 가져오기(중간에 다른 태그가 있어도 가능!)

[<a class="framework"> tensorflow </a>,
 <a class="framework"> pytorch </a>,
 <a class="framework"> spring </a>]

In [17]:
bs.select('div > a') #div태그 바로 아래에 있는 a태그 가져오기(직접 하위관계의 태그만 가져옴)
bs.select('div > div') #div태그 바로 아래에 있는 div태그 가져오기

[<div class="data1">
 <span class="language"> c++ </span>
 <span class="language"> java </span>
 <span class="language"> python </span>
 </div>,
 <div class="data2">
 <a class="framework"> tensorflow </a>
 <a class="framework"> pytorch </a>
 <a class="framework"> spring </a>
 </div>]

#### 네이버 뉴스

In [18]:
!pip install psycopg2

In [19]:
import pandas as pd
import requests
import psycopg2
from tqdm import tqdm

In [20]:
response = requests.get(
    'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=023&date=20220820&page=1000',
    headers ={'User-Agent': 'Mozilla 5.0'})
bs = BeautifulSoup(response.text, 'lxml')

In [21]:
_last_page_num = bs.select('div.paging > strong')
last_page_num = int(_last_page_num[0].text)

In [22]:
last_page_num

7

In [59]:
[tag.text.replace('\n','').replace('\t','').strip() for tag in bs.select('li > dl > dt > a') if tag.text != '\n\n']

[]

제목 추출

In [24]:
all_titles = []
for page in range(1, last_page_num+1):
    response = requests.get(
        f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=023&date=20220820&page={page}',
        headers ={'User-Agent': 'Mozilla 5.0'})
    bs = BeautifulSoup(response.text, 'lxml')
    titles = [tag.text.replace('\n','').replace('\t','').strip() for tag in bs.select('li > dl > dt > a') if tag.text != '\n\n']
    all_titles += titles
all_titles

['[NOW] ‘카톡 손주’ 덕분에 스마트폰·키오스크 혼자서도 잘 써요',
 '저소득층 의료비 최대 5000만원 지원... 적용대상도 6대중증→모든 질환',
 '두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽',
 '월성·북송… 대통령기록관 하루 두번 압수수색',
 '워터파크서 8분간 물에 빠진 아이…학원선생도 안전요원도 몰랐다',
 '손흥민 슛 골대 강타… 1분뒤 케인이 결승골 쐈다',
 '늦게 비켜줬다고 구급차 운전자가 손가락 욕? [영상]',
 '경찰에 체포된 美여성…손목에 찬 수갑 풀고 총 쐈다',
 '우상호 “尹정부 무능에 믿을 건 민주당밖에 없다는 인식 확산”',
 '“열기에 생새우 익었다”…中폭염 탓에 이런 일도',
 '이재명, 전북 경선서도 압승…누적 득표율 78.05%',
 '“악이 판치는 세상” 정경심 형집행정지 불허에 野 반발',
 '이재명 “계파정치는 상상 못해… 공정하게 국민·당원 선택 받을 수 있도록 할 것”',
 '“조종사가 깜빡 졸아”…활주로 지나쳐 날아간 에티오피아 비행기',
 '서울서 여성전용주차장 사라진다...가족우선주차장으로 전환',
 '박용진, 이재명 겨냥 “강성 지지자들, 설득하고 말리는 게 지도자 역할”',
 '박지원 “尹 제안 北이 거부할 거라 예측했다…정부 대응 적절”',
 '박수현 “尹대통령 기자회견, 이준석 확실히 정리하라는 尹心 전달한 것”',
 '바짝 마른 양쯔강…600년 전 불상 나타났다',
 '‘전 여친 폭행 혐의’ 긱스...퍼거슨 “긱스는 화낸 적 없어”',
 '조각가 박임향 첫 개인전 ‘스며듦’',
 '‘하트시그널3′ 출신 정비사 서민재 “남태현 필로폰 투약, 제 캐비넷 보세요”',
 '北매체 “尹 낮은 지지율은 정확한 민심… 보수층도 등돌려”',
 '장예찬 “우리 당에서 尹정부 애정 가진 2030 스피커 저밖에 없더라”',
 '방탄소년단, 美 MTV 비디오뮤직어워즈 6개 부문 후보',
 '“소품총 비극 책임자는…” 총 들었던 볼드윈이 지목한 두 사람',
 '검찰, ‘차량 화재’ 

In [25]:
def get_bs_data_from_naver_news(oid,date,page):
    response = requests.get(
        f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date={date}&page={page}',
        headers ={'User-Agent': 'Mozilla 5.0'})
    
    bs = BeautifulSoup(response.text, 'lxml')

    return bs
    

In [26]:
def get_titles_from_tags(bs: BeautifulSoup):
    return [tag.text.replace('\n','').replace('\t','').strip() for tag in bs.select('li > dl > dt > a') if tag.text != '\n\n']

In [27]:
bs = get_bs_data_from_naver_news('023','20220820',1000)
_last_page_num = bs.select('div.paging > strong')
last_page_num = int(_last_page_num[0].text)


In [28]:
all_titles = []
for page in range(1, last_page_num+1):
    response = get_bs_data_from_naver_news('023', '20220820', page)
    titles = get_titles_from_tags(response)
    all_titles += titles
all_titles

['[NOW] ‘카톡 손주’ 덕분에 스마트폰·키오스크 혼자서도 잘 써요',
 '저소득층 의료비 최대 5000만원 지원... 적용대상도 6대중증→모든 질환',
 '두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽',
 '월성·북송… 대통령기록관 하루 두번 압수수색',
 '워터파크서 8분간 물에 빠진 아이…학원선생도 안전요원도 몰랐다',
 '손흥민 슛 골대 강타… 1분뒤 케인이 결승골 쐈다',
 '늦게 비켜줬다고 구급차 운전자가 손가락 욕? [영상]',
 '경찰에 체포된 美여성…손목에 찬 수갑 풀고 총 쐈다',
 '우상호 “尹정부 무능에 믿을 건 민주당밖에 없다는 인식 확산”',
 '“열기에 생새우 익었다”…中폭염 탓에 이런 일도',
 '이재명, 전북 경선서도 압승…누적 득표율 78.05%',
 '“악이 판치는 세상” 정경심 형집행정지 불허에 野 반발',
 '이재명 “계파정치는 상상 못해… 공정하게 국민·당원 선택 받을 수 있도록 할 것”',
 '“조종사가 깜빡 졸아”…활주로 지나쳐 날아간 에티오피아 비행기',
 '서울서 여성전용주차장 사라진다...가족우선주차장으로 전환',
 '박용진, 이재명 겨냥 “강성 지지자들, 설득하고 말리는 게 지도자 역할”',
 '박지원 “尹 제안 北이 거부할 거라 예측했다…정부 대응 적절”',
 '박수현 “尹대통령 기자회견, 이준석 확실히 정리하라는 尹心 전달한 것”',
 '바짝 마른 양쯔강…600년 전 불상 나타났다',
 '‘전 여친 폭행 혐의’ 긱스...퍼거슨 “긱스는 화낸 적 없어”',
 '조각가 박임향 첫 개인전 ‘스며듦’',
 '‘하트시그널3′ 출신 정비사 서민재 “남태현 필로폰 투약, 제 캐비넷 보세요”',
 '北매체 “尹 낮은 지지율은 정확한 민심… 보수층도 등돌려”',
 '장예찬 “우리 당에서 尹정부 애정 가진 2030 스피커 저밖에 없더라”',
 '방탄소년단, 美 MTV 비디오뮤직어워즈 6개 부문 후보',
 '“소품총 비극 책임자는…” 총 들었던 볼드윈이 지목한 두 사람',
 '검찰, ‘차량 화재’ 

In [29]:
def get_body_urls_from_tags(bs: BeautifulSoup):
    return[tag.attrs.get('href') for tag in bs.select('li > dl > dt > a') if tag.text != '\n\n']

In [30]:
all_titles = []
all_urls = []
for page in tqdm(range(1, last_page_num+1)):
    response = get_bs_data_from_naver_news('023', '20220820', page)
    titles = get_titles_from_tags(response)
    urls = get_body_urls_from_tags(response)
    all_titles += titles
    all_urls += urls


100%|██████████| 7/7 [00:01<00:00,  6.40it/s]


URL 추출

In [31]:
bs.select('li > dl > dt > a')[0].attrs.get('href') #딕셔너리 형태로 변환 키값으로 본문링크 추출

'https://n.news.naver.com/mnews/article/023/0003711012'

In [32]:
get_body_urls_from_tags(bs)

['https://n.news.naver.com/mnews/article/023/0003711012',
 'https://n.news.naver.com/mnews/article/023/0003711011',
 'https://n.news.naver.com/mnews/article/023/0003711010',
 'https://n.news.naver.com/mnews/article/023/0003711009',
 'https://n.news.naver.com/mnews/article/023/0003711008',
 'https://n.news.naver.com/mnews/article/023/0003711007',
 'https://n.news.naver.com/mnews/article/023/0003711006',
 'https://n.news.naver.com/mnews/article/023/0003711005',
 'https://n.news.naver.com/mnews/article/023/0003711004',
 'https://n.news.naver.com/mnews/article/023/0003711003',
 'https://n.news.naver.com/mnews/article/023/0003711002',
 'https://n.news.naver.com/mnews/article/023/0003711001',
 'https://n.news.naver.com/mnews/article/023/0003711000',
 'https://n.news.naver.com/mnews/article/023/0003710999',
 'https://n.news.naver.com/mnews/article/023/0003710998',
 'https://n.news.naver.com/mnews/article/023/0003710997',
 'https://n.news.naver.com/mnews/article/023/0003710996']

본문 추출

In [33]:
def get_contents_from_url(url:str):
    response = requests.get(url, headers = {'User-Agent' : 'Mozilla 5.0'})
    bs = BeautifulSoup(response.text, 'lxml')

    return bs.select('div#dic_area')[0].text

In [34]:
get_contents_from_url('https://n.news.naver.com/mnews/article/023/0003710996')

'\n스티븐 킹과 시드니 셸던의 팬이었던 어린 시절, 큰 의문이 있었다. 어떻게 한 인간이 이 길고 복잡한 소설을 그토록 줄기차게 써낼 수 있는지 궁금했던 것이다. 작가가 됐지만 여전히 나는 동료 작가들에게 작법이나 창작의 비밀 같은 것을 묻는다. 소설이 써지지 않을 때는 “첫 문장을 썼으니 반은 쓴 거나 매한가지!”라는 작가들의 자조 섞인 농담을 위안 삼기도 한다.하지만 정작 내가 소설 쓰는 일에 대해 잘 말하지 않게 된 건 글 쓰는 일이 사람들이 흔히 생각하는 ‘뮤즈’나 ‘영감’과 무관한 일이라는 걸 알아버렸기 때문이다. 아이디어나 작법이 중요한 건 사실이다. 하지만 작품을 꾸준히 쓰려면 태도가 더 중요하다. 글 쓰는 걸 좋아하는 것과 글을 써서 밥을 먹고 사는 일은 전혀 다른 종류의 일이기 때문이다.내 경우 좋아서 쓰는 것이라기보다, 쓰지 않으면 견딜 수 없으니 쓴다. 언제 써질지 모르니 불안해서 쓰고, 앞으로는 쓸 수 없을 거란 예감에 시달리니 쓰지 않을 수 없다. 간절함은 자신이 가진 능력을 증폭시키는 힘이 있다. 배우 윤여정도 “가장 연기가 잘될 때는 돈이 없을 때다”라고 말하지 않았던가. 책상 앞에 앉으면 막막함에 불안이 차오르지만, 일단 5매만 쓰자, 오늘은 썼으니 내일도 쓸 수 있을 거다, 라는 생각으로 꾸역꾸역 쓴다. 하루하루 그런 시간들이 모여 책이 된다.가슴 설레는 일을 하라는 스티브 잡스의 말은 우리를 꿈꾸게 하지만 현실에선 잘 적용되지 않는다. 글이 쓰고 싶어서 아침마다 눈이 번쩍 떠진다는 작가를 본 적이 없다. 그러니 “영감을 기다리는 건 아마추어고, 우리는 그냥 일을 하러 간다”는 소설가 필립 로스의 말을 성경처럼 새기고 ‘영감’이 아닌 ‘마감’의 힘으로 버티는 게 이 업계의 일이다.삶의 많은 부분이 실은 이런 힘에 의해 움직인다. 그러니 할 수 있고, 갈 수 있고, 쓸 수 있을 때 힘 내보자는 생각이 든다. 생전 박완서 선생님이 나이가 드니 책 못 읽는 게 가장 힘들다고 하셨던 말이 생각난다. 나도 노안이 오면 곧 못 읽을 때가 

In [35]:
response = requests.get('https://n.news.naver.com/mnews/article/023/0003711001', headers = {'User-Agent' : 'Mozilla 5.0'})
bs = BeautifulSoup(response.text, 'lxml')

In [36]:
bs.select('div#dic_area')[0].text

'\n19세때 장기기증 이태경군 아버지… 장기기증 서약한 고교생들 만났다\n\n\n\n지난 6일 오후 서울 서대문구 사랑의장기기증운동본부에서 2010년 뇌사 판정을 받은 후 7명에게 장기를 나눠주고 세상을 떠난 이태경군의 아버지 이대호(가운데)씨가 최근 장기기증 희망 등록을 한 고등학생 4명으로부터 꽃다발을 받고 있다. 이씨는“남을 돕는 것을 무척 좋아하던 태경이도 하늘나라에서 기뻐하고 있을 것”이라고 말했다./사랑의장기기증운동본부\t\t\t\t\t\t\t\t\t\t“장기를 기증한 아들에게 ‘대견하다’는 말을 직접 해주지 못했어요. 대신 여러분에게 오늘 그 말을 전해주고 싶습니다.”지난 6일 오후 3시쯤 서울 서대문구 사랑의장기기증운동본부 회의실. 이 말을 하는 이대호(60)씨의 눈시울이 붉어졌다. 이씨의 아들 이태경군은 열아홉이던 지난 2010년 8월 뇌와 척수에 종양이 생겨 뇌사 판정을 받았다. 결국 이군은 세상을 떠났지만 장기를 기증해 7명의 생명을 살렸다. 그 후로 12년이 지났다. 아버지 이씨는 사랑의장기기증운동본부 제안으로 지난 6일 당시 아들 또래인 고교생 4명을 만나 당시를 떠올리는 시간을 가졌다. 모두 만일의 사태가 생기면 ‘장기 기증을 하겠다’고 서약한 학생들이다.지난 2010년 당시 서울 송파구 가락시장에서 홀로 작은 채소 가게를 하던 이씨에게 아들은 든든한 지원군이자 둘도 없는 친구였다. 하지만 2010년 7월 어느 날, 학교에서 돌아온 아들이 “요즘 한쪽 눈이 잘 보이지 않아 학교에서 칠판 보기 불편하다”고 했다. 얼마 후 한 대학 병원에서 들은 검사 결과는 충격적이었다. ‘신경 교종증’이란 들어보지도 못한 병이었다. 뇌와 척수 내부에 있는 신경교세포에 종양이 생겼고, 이 종양이 시신경을 누르고 있어 한쪽 눈이 잘 보이지 않는 증상이 나타난 것이라고 했다. 이씨는 곧바로 병원에 입원시켜 치료를 시작했지만 태경군은 다음 날 의식을 잃고 쓰러졌다. 일주일 뒤 태경군은 의사에게 뇌사 판정을 받았다.\n\n\n\n故 이태경군\t\t\t\t\t\

데이터 통합

In [37]:
news = pd.DataFrame({
    'date': '20220820',
    'oid' : '023',
    'title': all_titles,
    'url': all_urls})

In [38]:
for row in tqdm(news.itertuples()):
    try:
        news.loc[row.Index, 'contents'] = get_contents_from_url(row.url)
    except:
        pass

141it [00:27,  5.16it/s]


In [39]:
news

,date,oid,title,url,contents
0,20220820,023,[NOW] ‘카톡 손주’ 덕분에 스마트폰·키오스크 혼자서도 잘 써요,https://n.news.naver.com/mnews/article/023/000...,\n노인·청년 이어주는 디지털 교육 인기\t\t\t\t\t\t\t경기 남양주에 사는...
1,20220820,023,저소득층 의료비 최대 5000만원 지원... 적용대상도 6대중증→모든 질환,https://n.news.naver.com/mnews/article/023/000...,\n尹대통령 “정치복지서 약자복지로”\n\n\n\n윤석열 대통령이 19일 용산 대통...
2,20220820,023,"두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽",https://n.news.naver.com/mnews/article/023/000...,"\n보훈처, 前광복회장 두번째 고발\n\n\n\n\t\t\t\t\t\t\t\t\t\..."
3,20220820,023,월성·북송… 대통령기록관 하루 두번 압수수색,https://n.news.naver.com/mnews/article/023/000...,\n文정부 청와대 윗선으로 향하는 검찰 수사\t\t\t\t\t\t\t검찰이 19일 ...
4,20220820,023,워터파크서 8분간 물에 빠진 아이…학원선생도 안전요원도 몰랐다,https://n.news.naver.com/mnews/article/023/000...,\n\n\n\n\n지난 6월25일 강원도 홍천의 한 워터파크 파도풀에 8살 어린이가...
...,...,...,...,...,...
136,20220820,023,“엄마가 내 어린시절 노출해 큰 피해 봤어요” 일본서 셰어런팅 역풍,https://n.news.naver.com/mnews/article/023/000...,\n\n\n\n\n'셰어런팅' 논란으로 역풍을 맞고 있는 일본 사이바라 리에코(58...
137,20220820,023,"두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽",https://n.news.naver.com/mnews/article/023/000...,"\n보훈처, 前광복회장 두번째 고발\n\n\n\n\t\t\t\t\t\t\t\t\t\..."
138,20220820,023,월성·북송… 대통령기록관 하루 두번 압수수색,https://n.news.naver.com/mnews/article/023/000...,\n文정부 청와대 윗선으로 향하는 검찰 수사\t\t\t\t\t\t\t검찰이 19일 ...
139,20220820,023,"갤Z폴드·Z플립, 이번에도 우크라에선 ‘Z’ 빠졌다",https://n.news.naver.com/mnews/article/023/000...,\n\n\n\n\n삼성전자 우크라이나 홈페이지에 게시된 갤럭시Z폴드4와 갤럭시Z플립...


DB 연결

In [49]:
news.to_sql(
    'news', #table 이름
    engine,
    if_exists = 'append',
    index = True
)

2022-08-21 10:35:25,968 INFO sqlalchemy.engine.Engine select version()
2022-08-21 10:35:25,970 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-08-21 10:35:25,974 INFO sqlalchemy.engine.Engine select current_schema()
2022-08-21 10:35:25,975 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-08-21 10:35:25,977 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-08-21 10:35:25,978 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-08-21 10:35:25,981 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-08-21 10:35:25,981 INFO sqlalchemy.engine.Engine [generated in 0.00067s] {'name': 'news'}
2022-08-21 10:35:25,989 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-21 10:35:25,991 INFO sqlalchemy.engine.Engine INSERT INTO news (index, date, oid, title, url, contents) VALUES (%(index)s, %(date)s, %(oid)s, %(title)s, %(url)s, %(contents)s)
2022-08-21 10:35:25

In [50]:
pd.read_sql('SELECT * FROM news', engine)

2022-08-21 10:35:29,004 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-08-21 10:35:29,005 INFO sqlalchemy.engine.Engine [cached since 3.024s ago] {'name': 'SELECT * FROM news'}
2022-08-21 10:35:29,009 INFO sqlalchemy.engine.Engine SELECT * FROM news
2022-08-21 10:35:29,010 INFO sqlalchemy.engine.Engine [raw sql] {}


,index,date,oid,title,url,contents
0,0,20220820,023,[NOW] ‘카톡 손주’ 덕분에 스마트폰·키오스크 혼자서도 잘 써요,https://n.news.naver.com/mnews/article/023/000...,\n노인·청년 이어주는 디지털 교육 인기\t\t\t\t\t\t\t경기 남양주에 사는...
1,1,20220820,023,저소득층 의료비 최대 5000만원 지원... 적용대상도 6대중증→모든 질환,https://n.news.naver.com/mnews/article/023/000...,\n尹대통령 “정치복지서 약자복지로”\n\n\n\n윤석열 대통령이 19일 용산 대통...
2,2,20220820,023,"두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽",https://n.news.naver.com/mnews/article/023/000...,"\n보훈처, 前광복회장 두번째 고발\n\n\n\n\t\t\t\t\t\t\t\t\t\..."
3,3,20220820,023,월성·북송… 대통령기록관 하루 두번 압수수색,https://n.news.naver.com/mnews/article/023/000...,\n文정부 청와대 윗선으로 향하는 검찰 수사\t\t\t\t\t\t\t검찰이 19일 ...
4,4,20220820,023,워터파크서 8분간 물에 빠진 아이…학원선생도 안전요원도 몰랐다,https://n.news.naver.com/mnews/article/023/000...,\n\n\n\n\n지난 6월25일 강원도 홍천의 한 워터파크 파도풀에 8살 어린이가...
...,...,...,...,...,...,...
277,136,20220820,023,“엄마가 내 어린시절 노출해 큰 피해 봤어요” 일본서 셰어런팅 역풍,https://n.news.naver.com/mnews/article/023/000...,\n\n\n\n\n'셰어런팅' 논란으로 역풍을 맞고 있는 일본 사이바라 리에코(58...
278,137,20220820,023,"두 배 비싸게 만든 김원웅식 위인전… 김구 290쪽, 모친은 430쪽",https://n.news.naver.com/mnews/article/023/000...,"\n보훈처, 前광복회장 두번째 고발\n\n\n\n\t\t\t\t\t\t\t\t\t\..."
279,138,20220820,023,월성·북송… 대통령기록관 하루 두번 압수수색,https://n.news.naver.com/mnews/article/023/000...,\n文정부 청와대 윗선으로 향하는 검찰 수사\t\t\t\t\t\t\t검찰이 19일 ...
280,139,20220820,023,"갤Z폴드·Z플립, 이번에도 우크라에선 ‘Z’ 빠졌다",https://n.news.naver.com/mnews/article/023/000...,\n\n\n\n\n삼성전자 우크라이나 홈페이지에 게시된 갤럭시Z폴드4와 갤럭시Z플립...


datetime 형식

In [51]:
from datetime import datetime
datetime.strptime('20220820','%Y%m%d') #Y-2022 / y-22/ m-월(숫자)정보/ d-날짜(숫자)정보

datetime.datetime(2022, 8, 20, 0, 0)

In [52]:
news.date = news.date.apply(lambda x: datetime.strptime(x,'%Y%m%d'))
#apply : 열이나 행단위로 적용
#x : 로우값 하나하나

In [54]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      141 non-null    datetime64[ns]
 1   oid       141 non-null    object        
 2   title     141 non-null    object        
 3   url       141 non-null    object        
 4   contents  128 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 5.6+ KB


In [55]:
news.to_sql(
    'news',
    engine,
    if_exists = 'replace',
    index = True
)

2022-08-21 10:37:20,267 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-08-21 10:37:20,270 INFO sqlalchemy.engine.Engine [cached since 114.3s ago] {'name': 'news'}
2022-08-21 10:37:20,275 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-08-21 10:37:20,276 INFO sqlalchemy.engine.Engine [cached since 114.3s ago] {'name': 'news'}
2022-08-21 10:37:20,280 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2022-08-21 10:37:20,280 INFO sqlalchemy.engine.Engine [generated in 0.00054s] {'schema': 'public'}
2022-08-21 10:37:20,283 INFO sqlalchemy.engine.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
          

#### 네이버 증권

In [100]:
pd.read_html('https://finance.naver.com/sise/sise_market_sum.naver', encoding='cp949')[1].dropna(subset = ['종목명'])

,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
1,1.0,삼성전자,60900.0,600.0,-0.98%,100.0,3635598.0,5969783.0,49.81,6882431.0,9.24,13.92,NaN
2,2.0,LG에너지솔루션,447000.0,5000.0,-1.11%,500.0,1045980.0,234000.0,4.07,223306.0,742.52,10.68,NaN
3,3.0,SK하이닉스,96400.0,700.0,+0.73%,5000.0,701794.0,728002.0,49.95,1880608.0,6.11,16.84,NaN
4,4.0,삼성바이오로직스,868000.0,7000.0,-0.80%,2500.0,617790.0,71174.0,10.77,29419.0,115.44,8.21,NaN
5,5.0,삼성전자우,55800.0,600.0,-1.06%,100.0,459171.0,822887.0,71.92,463075.0,8.46,NaN,NaN
9,6.0,LG화학,640000.0,11000.0,-1.69%,5000.0,451791.0,70592.0,47.82,124628.0,22.56,18.47,NaN
10,7.0,삼성SDI,618000.0,6000.0,-0.96%,5000.0,424965.0,68765.0,43.77,124151.0,28.83,8.45,NaN
11,8.0,NAVER,248000.0,2000.0,-0.80%,100.0,406842.0,164049.0,53.25,371000.0,40.46,106.72,NaN
12,9.0,현대차,190000.0,4000.0,+2.15%,5000.0,405970.0,213668.0,27.98,765888.0,8.43,6.84,NaN
13,10.0,카카오,76700.0,2500.0,-3.16%,100.0,341430.0,445150.0,28.85,1783167.0,15.34,17.10,NaN


종목명

In [63]:
response = requests.get(
    'https://finance.naver.com/item/main.naver?code=005930',
    headers = {'User-Agent': 'Mozilla 5.0'})
bs = BeautifulSoup(response.text, 'lxml')
bs.select('div > h2 > a')[0].text


'삼성전자'

현재가

In [83]:
response = requests.get(
    'https://finance.naver.com/item/main.naver?code=005930',
    headers = {'User-Agent': 'Mozilla 5.0'})
bs = BeautifulSoup(response.text, 'lxml')
bs.select('div > p > em >span')[0].text

'60,900'

시가총액

In [ ]:
response = requests.get(
    'https://finance.naver.com/item/main.naver?code=005930',
    headers = {'User-Agent': 'Mozilla 5.0'})
bs = BeautifulSoup(response.text, 'lxml')
bs.select('div > p > em >span')[0].text

In [ ]:
titles = [tag.text.replace('\n','').replace('\t','').strip() for tag in bs.select('li > dl > dt > a') if tag.text != '\n\n']

#### 다음 증권

In [75]:
response = requests.get('https://finance.daum.net/api/trend/market_capitalization?page=1&perPage=30&fieldName=marketCap&order=desc&market=KOSPI&pagination=true',
    headers = {'User-Agent': 'Mozilla 5.0'})
bs = BeautifulSoup(response.text, 'lxml')
bs.select('div')


[]

데이터 url 주소로 접근

In [87]:
import json

In [85]:
response = requests.get('https://finance.daum.net/api/trend/market_capitalization?page=1&perPage=30&fieldName=marketCap&order=desc&market=KOSPI&pagination=true',
    headers ={'User-Agent': 'Mozilla/5.0',
    'referer' : 'https://finance.daum.net/domestic/market_cap'})

bs = BeautifulSoup(response.text, 'lxml')
bs.text

'{"data":[{"rank":1,"name":"삼성전자","symbolCode":"A005930","code":"KR7005930003","tradePrice":60900.0,"change":"FALL","changePrice":600.0,"changeRate":-0.0097560976,"marketCap":363559757295000.0,"listedShareCount":5969782550,"foreignRatio":"49.810"},{"rank":2,"name":"LG에너지솔루션","symbolCode":"A373220","code":"KR7373220003","tradePrice":447000.0,"change":"FALL","changePrice":5000.0,"changeRate":-0.0110619469,"marketCap":104598000000000.0,"listedShareCount":234000000,"foreignRatio":"4.070"},{"rank":3,"name":"SK하이닉스","symbolCode":"A000660","code":"KR7000660001","tradePrice":96400.0,"change":"RISE","changePrice":700.0,"changeRate":0.0073145246,"marketCap":70179427986000.0,"listedShareCount":728002365,"foreignRatio":"49.950"},{"rank":4,"name":"삼성바이오로직스","symbolCode":"A207940","code":"KR7207940008","tradePrice":868000.0,"change":"FALL","changePrice":7000.0,"changeRate":-0.008,"marketCap":61779032000000.0,"listedShareCount":71174000,"foreignRatio":"10.770"},{"rank":5,"name":"삼성전자우","symbolCode":"

In [89]:
pd.DataFrame(json.loads(bs.text).get('data'))

,rank,name,symbolCode,code,tradePrice,change,changePrice,changeRate,marketCap,listedShareCount,foreignRatio
0,1,삼성전자,A005930,KR7005930003,60900.0,FALL,600.0,-0.009756,3.635598e+14,5969782550,49.810
1,2,LG에너지솔루션,A373220,KR7373220003,447000.0,FALL,5000.0,-0.011062,1.045980e+14,234000000,4.070
2,3,SK하이닉스,A000660,KR7000660001,96400.0,RISE,700.0,0.007315,7.017943e+13,728002365,49.950
3,4,삼성바이오로직스,A207940,KR7207940008,868000.0,FALL,7000.0,-0.008000,6.177903e+13,71174000,10.770
4,5,삼성전자우,A005935,KR7005931001,55800.0,FALL,600.0,-0.010638,4.591708e+13,822886700,71.920
5,6,LG화학,A051910,KR7051910008,640000.0,FALL,11000.0,-0.016897,4.517910e+13,70592343,47.820
6,7,삼성SDI,A006400,KR7006400006,618000.0,FALL,6000.0,-0.009615,4.249648e+13,68764530,43.770
7,8,NAVER,A035420,KR7035420009,248000.0,FALL,2000.0,-0.008000,4.068417e+13,164049085,53.250
8,9,현대차,A005380,KR7005380001,190000.0,RISE,4000.0,0.021505,4.059696e+13,213668187,27.980
9,10,카카오,A035720,KR7035720002,76700.0,FALL,2500.0,-0.031566,3.414298e+13,445149626,28.850
